In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
os.chdir('../')
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

import sys
sys.path.append('examples')

import jhutil; jhutil.color_log(1111, )

# Don't Unfold!

# .

In [2]:
import json
from collections import OrderedDict
from pycolmap.scene_manager import Camera

def _load_cameras_son(self, json_file_path):
    self.cameras = OrderedDict()

    with open(json_file_path, 'r') as file:
        data = json.load(file)

    frames = data.get("frames", [])

    for frame_index, frame in enumerate(frames):
        file_path = frame.get("file_path", "")
        width = frame.get("w", 0)
        height = frame.get("h", 0)
        fl_x = frame.get("fl_x", 0.0)
        fl_y = frame.get("fl_y", 0.0)
        cx = frame.get("cx", 0.0)
        cy = frame.get("cy", 0.0)

        # Create a camera object with the necessary parameters
        camera_params = {
            "file_path": file_path,
            "width": width,
            "height": height,
            "fl_x": fl_x,
            "fl_y": fl_y,
            "cx": cx,
            "cy": cy,
        }
        
        camera_id = frame_index + 1
        self.cameras[camera_id] = camera_params
    self.last_camera_id = camera_id


In [ ]:
from pycolmap.scene_manager import SceneManager

colmap_dir = "data/360_v2/garden/sparse/0/"
from pycolmap.scene_manager import Camera


manager = SceneManager(colmap_dir)
manager.load_cameras()
manager.load_images()
manager.load_points3D()

In [ ]:
from pycolmap.scene_manager import SceneManager


    
colmap_dir = "data/360_v2/garden/sparse/0/"
from pycolmap.scene_manager import Quaternion, Image

def _load_points3D_bin(self, input_file):
    with open(input_file, 'rb') as f:
        num_points3D = struct.unpack('L', f.read(8))[0]

        self.points3D = np.empty((num_points3D, 3))
        self.point3D_ids = np.empty(num_points3D, dtype=np.uint64)
        self.point3D_colors = np.empty((num_points3D, 3), dtype=np.uint8)
        self.point3D_id_to_point3D_idx = dict()
        self.point3D_id_to_images = dict()
        self.point3D_errors = np.empty(num_points3D)

        data_struct = struct.Struct('<Q 3d 3B d Q')

        for i in range(num_points3D):
            data = data_struct.unpack(f.read(data_struct.size))
            self.point3D_ids[i] = data[0]
            self.points3D[i] = data[1:4]
            self.point3D_colors[i] = data[4:7]
            self.point3D_errors[i] = data[7]
            track_len = data[8]

            self.point3D_id_to_point3D_idx[self.point3D_ids[i]] = i

            data = struct.unpack(f'{2*track_len}I', f.read(2 * track_len * 4))

            self.point3D_id_to_images[self.point3D_ids[i]] = \
                np.array(data, dtype=np.uint32).reshape(track_len, 2)

SceneManager._load_points3D_bin = _load_points3D_bin

manager = SceneManager(colmap_dir)
# manager.load_cameras()
# manager.load_images()
manager.load_points3D()
from jhutil import color_log; color_log(1111, manager.point3D_id_to_images)

In [ ]:
# intrinsic
# fx, fy, cx, cy, camera_type, width, height가 필요함
dir(parser.manager.cameras[1])

In [71]:
import json
from collections import OrderedDict
from pycolmap.scene_manager import Quaternion, Image, Camera
import numpy as np


class SceneManagerDiva360:
    def __init__(self, json_path):
        self.json_path = json_path
        self.cameras = OrderedDict()
        self.images = OrderedDict()
        self.name_to_image_id = {}
        self.last_image_id = 0

    def load_cameras_from_json(self):
        with open(self.json_path, "r") as file:
            data = json.load(file)

        frames = data.get("frames", [])

        for camera_id, frame in enumerate(frames, 1):
            file_path = frame.get("file_path", "")
            width = frame.get("w", 0)
            height = frame.get("h", 0)
            fl_x = frame.get("fl_x", 0.0)
            fl_y = frame.get("fl_y", 0.0)
            cx = frame.get("cx", 0.0)
            cy = frame.get("cy", 0.0)

            # Create a Camera object
            camera = Camera(1, width, height, [fl_x, fl_y, cx, cy])

            # Store the Camera object along with the file path
            self.cameras[camera_id] = {"file_path": file_path, "camera": camera}

    def load_extrinsics_from_json(self):
        with open(self.json_path, "r") as file:
            data = json.load(file)

        frames = data.get("frames", [])

        for camera_id, frame in enumerate(frames, 1):
            file_path = frame.get("file_path", "")
            transform_matrix = np.array(frame.get("transform_matrix", []))

            if transform_matrix.shape == (4, 4):
                rotation_matrix = transform_matrix[:3, :3]
                translation = transform_matrix[:3, 3]

                # Convert rotation matrix to quaternion
                q = Quaternion.FromR(rotation_matrix)

                # Create an Image object
                image = Image(file_path, camera_id, q, translation)

                self.images[camera_id] = image
                self.name_to_image_id[file_path] = camera_id

                self.last_image_id = max(self.last_image_id, camera_id)

    def genrate_random_points3D(self, n=10000):
        translation_list = np.empty((len(self.images), 3))
        for camera_id, image in self.images.items():
            translation = image.tvec
            translation_list[camera_id - 1] = translation

        self.bbox = np.array(
            [np.min(translation_list, axis=0), np.max(translation_list, axis=0)]
        )
        self.points3D = np.random.uniform(self.bbox[0], self.bbox[1], (n, 3))
        self.point3D_colors = np.ones((n, 3), dtype=np.uint8) * 255


# Example usage
if __name__ == "__main__":
    json_path = "/data2/wlsgur4011/gsplat/data/Diva360_data/processed_data/penguin/dynamic_data/transforms_train.json"
    manager = SceneManagerDiva360(json_path)
    manager.load_cameras_from_json()
    manager.load_extrinsics_from_json()
    manager.genrate_random_points3D()

    for image_id, image in manager.images.items():
        camera = manager.cameras[image.camera_id]
